In [1]:
import torch

from modeling_dream.tokenization_dream import DreamTokenizer

/home/exx/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def select_device():
    if torch.cuda.is_available():
        return "cuda"
    mps_backend = getattr(torch.backends, "mps", None)
    if mps_backend is not None and mps_backend.is_available():
        return "mps"
    return "cpu"
# end

# --- Model Loading ---
model_path = "Dream-org/Dream-v0-Base-7B"
device = select_device()
dtype_by_device = {
    "cuda": torch.bfloat16,
    "mps": torch.float16,
    "cpu": torch.float32,
}
dtype = dtype_by_device[device]
print(f"Using device: {device} (dtype={dtype})")


Using device: cuda (dtype=torch.bfloat16)


In [3]:
tokenizer = DreamTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True,
    padding_side='left'
)

In [8]:
for k, v in tokenizer.special_tokens_map.items():
    print(f'{v}: {tokenizer.encode(v)}')
# end

<|beginoftext|>: [151665]
<|endoftext|>: [151643]
<|endoftext|>: [151643]
<|mask|>: [151666]
['<|beginoftext|>', '<|mask|>']: [151665, 151666]


[151666]

In [1]:
env

{'VSCODE_CWD': '/home/exx',
 'USER': 'exx',
 'VSCODE_NLS_CONFIG': '{"userLocale":"en","osLocale":"en","resolvedLanguage":"en","defaultMessagesFile":"/home/exx/.vscode/cli/servers/Stable-c9d77990917f3102ada88be140d28b038d1dd7c7/server/out/nls.messages.json","locale":"en","availableLanguages":{}}',
 'SHLVL': '1',
 'HOME': '/home/exx',
 'CONDA_SHLVL': '1',
 'HF_DATASETS_CACHE': '/data/hf_datasets_cache',
 'GTK_MODULES': 'gail:atk-bridge',
 'MANAGERPID': '2137',
 'SSL_CERT_FILE': '/usr/lib/ssl/cert.pem',
 'SYSTEMD_EXEC_PID': '2219',
 'GSM_SKIP_SSH_AGENT_WORKAROUND': 'true',
 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus',
 '_CE_M': '',
 '_CONDA_ROOT': '/home/exx/miniconda3',
 'DEBUGINFOD_URLS': 'https://debuginfod.ubuntu.com ',
 'LOGNAME': 'exx',
 'JOURNAL_STREAM': '8:26801',
 'MEMORY_PRESSURE_WATCH': '/sys/fs/cgroup/user.slice/user-1000.slice/user@1000.service/app.slice/vscode-tunnel.service/memory.pressure',
 'HF_HUB_CACHE': '/hf_hub_cache',
 '_CE_CONDA': '',
 'PATH': '/home/